In [1]:
import pandas as pd
import numpy as np

import copy
import itertools
import os
import re

from voting_lib import \
    ConvertResponsesToNumericVotes, \
    RepairVote, \
    PerformRankedChoiceVoting, \
    RemovePolicy, \
    RunBordaCountVote, \
    POLICY_NAMES, \
    PrintPolicyVec, \
    RunMultiRoundVoting

# Erroneous input (two policies ranked the same) is resolved randomly.
# Set the seed to prevent flaky voting.
np.random.seed(42)

In [2]:
print('The policies in order of votes are:')
print(POLICY_NAMES)

The policies in order of votes are:
['Policy A', 'Policy B', 'Policy C', 'Policy D']


In [3]:
working_dir = '/home/rgiordan/Documents/git_repos/Presentations/CCC_covid_vote'

In [4]:
date_string = '11-08_17_02_26'
raw_responses = pd.read_csv(os.path.join(working_dir, 
    'Children\'s_Community_Center_Out2022-' + date_string + '.csv'))

In [5]:
# Compile names
first_names = raw_responses[raw_responses.columns[1]].fillna('').to_numpy()
last_names = raw_responses[raw_responses.columns[2]].fillna('').to_numpy()
raw_full_names = np.array([ x + ' ' + y for x, y in zip(first_names, last_names) ])

In [6]:
staff_names = [
    'Allie Pollak',
    'Edna Tow',
    'Lara Gabato ',
    'Allyssa Adair',
    'Kathy Chew',
    'Ryan Farrell',
    'Malcolm Waugh',
    'Lilian Zakki',
    'laura McCaul',
    'Devaki  Nirula '
]

# Eliminate some rows by full name.
bad_names = ['Test Test']
keep_rows = [ name not in bad_names for name in raw_full_names ] 


# Note that the board decided to keep the one parent who didn't write their
# name rather than go through the trouble of trying to find them.
responses = raw_responses[keep_rows]
full_names = raw_full_names[keep_rows]
print(f'There were {len(full_names)} voters:\n\n', '\n'.join(full_names))


There were 54 voters:

 Ayako Hiwasa
Stefani Madril
Arielle Ryden
Jaffer Abbasi
Joe Graves
Chris Madril 
Matt Vander Sluis
Emily Schlessinger
Devaki  Nirula 
laura McCaul
Dora Zhang
Allie Pollak
Lilian Zakki
Jennifer  Oppeau
Malcolm Waugh
Juliana Monin
Harriet  Blackburn 
Edna Tow
Natanya Marks
Frances Schaeffer
Jennifer Nakata
Stevie Schwartz
ADELE SHEA
Roya Clune
Zach Alexander
Meghana Gadgil
Lara Gabato 
Sarah Chenoweth 
Theresa Keating
Jose Aranda
jessica thomas
Nathaniel Wolf
Lorian Schaeffer
Torrey Mansur
Jamie Greenwood
Allyssa Adair
Kathy Chew
Alli Beltz
ilyse magy
Miriam Wolodarski Lundberg
Mónica Henestroza
Shaye Mckenney 
Bren Darrow
Auddi Leos
Ian Wulfson
ADELE SHEA
Jill Fox
Dorothy  Thai
 
Ryan Farrell
Clare Armbruster
Encian  Pastel 
Xanh  Văn Ginzburg 
Stephanie and Pete Guinosso


In [7]:
# Get the indices of staff responses for separate analysis.
staff_inds = []
for staff_name in staff_names:
    inds = np.argwhere(full_names == staff_name).flatten()
    if len(inds) == 0:
        print(f'{staff_name} not found')
        assert(False)
    staff_inds.append(inds[0])

assert(np.all(full_names[staff_inds] == staff_names))

In [8]:
original_votes = ConvertResponsesToNumericVotes(responses)

In [9]:
# Fix malformed votes.
votes = copy.copy(original_votes)
for voter in range(votes.shape[1]):
    votes[:, voter] = RepairVote(votes[:, voter])
    
# Report on any votes that were modified.
for voter in range(votes.shape[1]):
    if np.any(votes[:, voter] != original_votes[:, voter]):
        print('\n==============================================')
        print(f'\nThe vote for voter {voter} was repaired:')
        PrintPolicyVec(original_votes[:, voter]) 
        print('...became...')
        PrintPolicyVec(votes[:, voter]) 
        print('\nThe process was as follows:')
        RepairVote(original_votes[:, voter], verbose=True)



The vote for voter 18 was repaired:
Policy A  Policy B  Policy C  Policy D  
3         2         2         1         
...became...
Policy A  Policy B  Policy C  Policy D  
4         3         2         1         

The process was as follows:
Original vote: [3 2 2 1]
Rank 1.  Current vote: [3 2 2 1]
Rank 2.  Current vote: [3 2 2 1]
Rank 2 duplicated, randomly splitting indices [1 2]
Rank 3.  Current vote: [4 3 2 1]
Rank 4.  Current vote: [4 3 2 1]
Final vote:
Policy A  Policy B  Policy C  Policy D  
4         3         2         1         


The vote for voter 50 was repaired:
Policy A  Policy B  Policy C  Policy D  
3         4         3         1         
...became...
Policy A  Policy B  Policy C  Policy D  
2         4         3         1         

The process was as follows:
Original vote: [3 4 3 1]
Rank 1.  Current vote: [3 4 3 1]
Rank 2.  Current vote: [3 4 3 1]
Rank 2 missing, decrementing other votes
Rank 2.  Current vote: [2 3 2 1]
Rank 2 duplicated, randomly splitting indice

In [10]:
print('Results for the whole school:')

RunMultiRoundVoting(votes)

Results for the whole school:


NameError: name 'RunMultiRoundVoting' is not defined

In [ ]:
print('We agreed to instant runoff voting, not Borda count.')

print('However, in case you are curious, the Borda count results is as follows (lower is better):')
borda_result = RunBordaCountVote(votes)
PrintPolicyVec(borda_result)



In [ ]:
# Run the auction separately for the staff who voted

print('Results only with staff ', ', '.join(staff_names))

RunMultiRoundVoting(votes[:, staff_inds])


In [ ]:
RunBordaCountVote(votes) # Lowest wins

In [ ]:
# Compile which polcies are indicated as unacceptable.
#print(responses.columns[7])
unacceptables = responses[responses.columns[7]].fillna('').to_numpy()

unacceptable_inds = []
for policy in [ 'A', 'B', 'C', 'D' ]:
    unacceptable_bools = [ re.match(f'Policy {policy}', v) is not None for v in unacceptables ]
    unacceptable_inds.append(np.argwhere(unacceptable_bools).flatten())

print('Count of unacceptables: ')
PrintPolicyVec([ len(v) for v in unacceptable_inds ])



In [ ]:
print(responses.columns[8])
unacceptable_text = responses[responses.columns[8]].fillna('').to_numpy()
for policy_ind in range(4):
    print(f'\n======================================================\n' + 
          f'Responses for {POLICY_NAMES[policy_ind]} unacceptable:')
    for voter_ind in unacceptable_inds[policy_ind]:
        print(f'\n > {full_names[voter_ind]} (index {voter_ind})')
        print(unacceptable_text[voter_ind])

print(f'\n======================================================\n' + 
f'Text entered with no policy marked unacceptable')

# Print text for folks who did not indicate that anything was unacceptable
for voter_ind in np.setdiff1d(np.arange(votes.shape[1]), np.hstack(unacceptable_inds)):
    if unacceptable_text[voter_ind] != '':
        print(f'\n > {full_names[voter_ind]} (index {voter_ind})')
        print(unacceptable_text[voter_ind])

In [ ]:
# Display other text
print(responses.columns[9])

comment_text = responses[responses.columns[9]].fillna('').to_numpy()

for voter_ind in range(votes.shape[1]):
    if comment_text[voter_ind] != '':
        print(f'\n > {full_names[voter_ind]} (index {voter_ind})')
        print(comment_text[voter_ind])